# Write-up and code for Jan 30

## To Do
- ~~Implement Black-Scholes formulas for European Call/Put Pricing~~
- ~~Implement standard binary tree/grid-based numerical algorithm for American Option Pricing and ensure it validates against Black-Scholes formula for Europeans~~
- Implement Longstaff-Schwartz Algorithm and ensure it validates against binary tree/grid-based solution for path-independent options
- Explore/Discuss an Approximate Dynamic Programming solution as an alternative to Longstaff-Schwartz Algorithm

## Black-Scholes formulas
According the the Black-Scholes Equation the price of an option is described by
$$ 
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2S^2\frac{\partial^2V}{\partial S^2} + rS\frac{\partial V}{\partial S} - rV = 0
$$
Where
- $S(t)$ is the price of the underlying asset
- $V(S,t)$ is the price of the option
- $\sigma$ is the standard deviation of the return for the underlying asset
- $r$ is the risk-free interest rate

Solving this equation for a European Call Option gives us
$$
\begin{align}
C(S_t, t) & = N(d_1) S_t - N(d_2) PV(K)\\
d_1 & = \frac{1}{\sigma\sqrt{T-t}}\Big[ ln\big( \frac{S_t}{K}\big) + \big( r + \frac{\sigma^2}{2}\big)(T-t)\big) \Big]\\
d_2 & = d_1 - \sigma\sqrt{T-t}\\
PV(K) & = Ke^{-r(T-t)}
\end{align}
$$

Using the put-call parity then gives us the price of a European Put Option

$$
\begin{align}
P(S_t,t) & = Ke^{-r(T-t)} - S_t + C(S_t,t)\\
& = N(-d_2)Ke^{-r(T-t)} - N(-d1)S_t
\end{align}
$$

Where
- $N(\cdot)$ is the cdf for a standard normal distribution
- $K$ is the strike price
- $T-t$ is the time left to maturity

The following code finds the price using the equations above:

In [7]:
from typing import NamedTuple

class EuropeanOption(NamedTuple):
    call: bool # true if call option, false if put
    S: float # underlying asset price
    K: float # strike price
    sigma: float # standard deviation of the return for the stock price
    tau: float # time to maturity (T-t), in years
    r: float # annual interest rate

In [53]:
import numpy as np
from scipy.stats import norm

def Black_Scholes_Price(option: EuropeanOption) -> float:
    # function to calculate the Black-Scholes price for a European Option
    if option.call:
        # return the price of a european call option
        return norm.cdf(d1(option))*option.S - norm.cdf(d2(option))*PV(option)*option.K
    
    return norm.cdf(-d2(option))*PV(option)*option.K - norm.cdf(-d1(option))*option.S


def d1(option: EuropeanOption) -> float:
    # function to calculate the intermediate value d1 in the Black-Scholes formula
    return 1/(option.sigma * np.sqrt(option.tau))*(np.log(option.S/option.K) + (option.r + option.sigma**2/2)*option.tau)
    
    
def d2(option: EuropeanOption) -> float:
    # function to calculate the intermediate value d2 in the Black-Scholes formula
    return d1(option) - option.sigma*np.sqrt(option.tau)


def PV(option: EuropeanOption) -> float:
    # function to calculate the discount factor
    return np.exp(-option.r*option.tau)

In [55]:
euro_call = EuropeanOption(True, 100, 110, 0.25, 0.5, 0.05)
euro_put = EuropeanOption(False, 100, 110, 0.25, 0.5, 0.05)
print("Price for European call: ", Black_Scholes_Price(euro_call))
print("Price for European put: ", Black_Scholes_Price(euro_put))

Price for European call:  4.22578239296007
Price for European put:  11.509872716076657


## Binary Tree implementation for pricing of American Options
We will construct a binary tree which can then be used to recursively find the price of an American Option.
- We start with an initial stock price of $S_0$
- At each time step $t$ we can either move up to $S_t = S_{t-1} \cdot u$ or down to $S_t = S_{t-1}\cdot d$. Where $u = e^{\sigma\sqrt{\Delta t}}$ and $d = e^{-\sigma\sqrt{\Delta t}} = \frac{1}{u}$
- At maturity the price of the option is equal to the payoff, $\max(S_T - K, 0)$ for a call option, and $\max(K-S_T,0)$ for a put option
- For an American option, we have that at each preceding state the value of the option is $\max(\textit{binomial value, exercise value})$, where binomial value is the discounted expected value of the next state, i.e. $C(S_t, t) = \gamma\mathbb E[C(S_{t+1},t+1)~|~S_t]$
- The probability of an upwards movement is given by $p = \frac{e^{(\mu-q)\Delta t}  - d}{u - d}$ where $q$ is the dividend yield for the underlying asset

In [50]:
class Option(NamedTuple):
    call: bool # true if call option, false if put
    american: bool # true if american option, false if european
    S: float # underlying asset price
    K: float # strike price
    sigma: float # standard deviation of the return for the stock price
    tau: float # time to maturity (T-t), in years
    r: float # annual interest rate
    q: float #dividend yield

In [ ]:
def Binary_Tree(option: Option, steps: int) -> float:
    # function that recursively finds the price of an American option
    assert(steps >= 0)
    
    delta_t = option.tau / steps
    up = np.exp(option.sigma*np.sqrt(delta_t))
    down = 1 / up
    # probability of upwards movement
    p = (np.exp((option.r - option.q)*delta_t)- down)/(up - down)
    
    # calculate the discount rate per time step
    gamma = np.exp(-option.r*delta_t)
    
    return binary_tree_helper(gamma, p, option.S, up, down, option, steps)


def binary_tree_helper(gamma: float, p: float, s: float, up: float, down: float, option: Option, steps: int) -> float:
    # helper function to calculate the option price
    # feed in the price
    if steps <= 0:
        # we have reached the end
        if option.call:
            return np.maximum(s-option.K, 0)
        return np.maximum(option.K - s, 0)
    
    # recursively find the price of the next step
    price_up = binary_tree_helper(gamma, p, s*up, up, down, option, steps - 1)
    price_down = binary_tree_helper(gamma, p, s*down, up, down, option, steps - 1)
    
    if option.american:
        # check if option is american
        if option.call:
            # check if call option
            return np.maximum(gamma*(p*price_up + (1-p)*price_down), s - option.K)
        return np.maximum(gamma*(p*price_up + (1-p)*price_down), option.K - s)
    
    # if it reaches here it is a european option
    return gamma*(p*price_up + (1-p)*price_down)

In [64]:
euro_call = Option(True, False, 100, 110, 0.25, 0.5, 0.05, 0)
euro_put = Option(False, False, 100, 110, 0.25, 0.5, 0.05, 0)
print("Price for European call: ", Binary_Tree(euro_call, 10))
print("Price for European put: ", Binary_Tree(euro_put, 10))

Price for European call:  4.207057860244597
Price for European put:  11.491148183361096


As we can see, using a binary grid converges quickly and produces roughly the same answer as Black-Scholes using only 10 time-steps between now and maturity.

In [65]:
am_call = Option(True, True, 100, 110, 0.25, 0.5, 0.05, 0)
am_put = Option(False, True, 100, 110, 0.25, 0.5, 0.05, 0)
print("Price for American call: ", Binary_Tree(am_call, 10))
print("Price for American put: ", Binary_Tree(am_put, 10))

Price for American call:  4.207057860244597
Price for American put:  12.102604663545703


We see that for an American put the price is slightly higher, than the price of the corresponding European put. The American Call produces the same price as the European Call, I don't know if this makes sense or not.

## Longstaff-Schwartz Algorithm
